In [2]:
#Find a way to get this to loop throughout Understat's website!

In [42]:
import pandas as pd
import numpy as np
from sklearn import linear_model
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import json
import os

In [20]:
result = requests.get('https://understat.com/league/EPL')
src = result.content
soup = BeautifulSoup(src, 'html.parser')
Misa = soup.find_all("script")

In [21]:
Game_Table_Player_data = []
for bruh in Misa[1:4]:
    bruh = str(bruh)
    s_1 = bruh.find('\'')
    s_2 = bruh.rfind('\'')
    bruh = bruh[s_1:s_2 + 1]
    Game_Table_Player_data.append(bruh)

In [34]:
Game_data = json.loads(Game_Table_Player_data[0].encode('ascii').decode('unicode-escape')[1:-1])
Table_data = json.loads(Game_Table_Player_data[1].encode('ascii').decode('unicode-escape')[1:-1])
Player_data  = json.loads(Game_Table_Player_data[2].encode('ascii').decode('unicode-escape')[1:-1])

Turning Game Data into a pd dataframe!

In [35]:
Game_ID = []
is_result = []
Home = []
Away = []
date_time = []
Home_Goals = []
Away_Goals = []
xG_Home = []
xG_Away = []
Prob_Home_Win = []
Prob_Draw = []
Prob_Away_Win = []
Game_data = json.loads(Game_Table_Player_data[0].encode('ascii').decode('unicode-escape')[1:-1])
for Game in Game_data:
    Game['h'] = Game['h']['title']
    Game['a'] = Game['a']['title']
    Game['Home Goals'] = Game['goals']['h']
    Game['Away Goals'] = Game['goals']['a']
    Game['xG Home'] = Game['xG']['h']
    Game['xG Away'] = Game['xG']['a']
    Game['Probability Home Win'] = Game['forecast']['w']
    Game['Probability Draw'] = Game['forecast']['d']
    Game['Probability Away Win'] = Game['forecast']['l']
    del Game['goals']
    del Game['xG']
    del Game['forecast']
    Game_ID.append(list(Game.values())[0])
    is_result.append(list(Game.values())[1])
    Home.append(list(Game.values())[2])
    Away.append(list(Game.values())[3])
    date_time.append(list(Game.values())[4])
    Home_Goals.append(list(Game.values())[5])
    Away_Goals.append(list(Game.values())[6])
    xG_Home.append(list(Game.values())[7])
    xG_Away.append(list(Game.values())[8])
    Prob_Home_Win.append(list(Game.values())[9])
    Prob_Draw.append(list(Game.values())[10])
    Prob_Away_Win.append(list(Game.values())[11])

In [36]:
PL_Game_Data_2021 = pd.DataFrame({'id': Game_ID, 'Is Result?': is_result, 'Home Team': Home, 
                                  'Away Team': Away, 'Home Goals': Home_Goals, 'Away Goals': Away_Goals,
                              'xG Home' : xG_Home, 'xG Away': xG_Away, 'Probability Home Win': Prob_Home_Win,
                                 'Probability Draw': Prob_Draw, 'Probability Away Win': Prob_Away_Win})

Team Data

In [49]:
#Fixing up the dictionary given by the parsing
Table_data = json.loads(Game_Table_Player_data[1].encode('ascii').decode('unicode-escape')[1:-1])
for key in list(Table_data.keys()):
    Table_data[Table_data[key]['title']] = Table_data[key]
    del Table_data[key]
for team in list(Table_data.keys()):
    Table_data[team] = Table_data[team]['history']

In [50]:
#initial dataset
for team in list(Table_data.keys()):
    h_a = []
    xG = []
    xGA = []
    npxG = []
    npxGA = []
    deep = []
    deep_allowed = []
    scored = []
    missed = []
    xpts = []
    result = []
    date = []
    wins = []
    draws = []
    loses = []
    pts = []
    npxGD = []
    ppda_att = []
    ppda_def = []
    ppda_allowed_att = []
    ppda_allowed_def = []
    for Game in Table_data[team]:
        Game['ppda att'] = Game['ppda']['att']
        Game['ppda def'] = Game['ppda']['def']
        Game['ppda allowed att'] = Game['ppda_allowed']['att']
        Game['ppda allowed def'] = Game['ppda_allowed']['def']
        del Game['ppda']
        del Game['ppda_allowed']
        h_a.append(list(Game.values())[0])
        xG.append(list(Game.values())[1])
        xGA.append(list(Game.values())[2])
        npxG.append(list(Game.values())[3])
        npxGA.append(list(Game.values())[4])
        deep.append(list(Game.values())[5])
        deep_allowed.append(list(Game.values())[6])
        scored.append(list(Game.values())[7])
        missed.append(list(Game.values())[8])
        xpts.append(list(Game.values())[9])
        result.append(list(Game.values())[10])
        date.append(list(Game.values())[11])
        wins.append(list(Game.values())[12])
        draws.append(list(Game.values())[13])
        loses.append(list(Game.values())[14])
        pts.append(list(Game.values())[15])
        npxGD.append(list(Game.values())[16])
        ppda_att.append(list(Game.values())[17])
        ppda_def.append(list(Game.values())[18])
        ppda_allowed_att.append(list(Game.values())[19])
        ppda_allowed_def.append(list(Game.values())[20])
    PL_Game_Data = pd.DataFrame({'Home/Away': h_a, 'xG': xG, 'xGA': xGA, 'non-penalty xG': npxG, 
                                     'non-penalty xG Against': npxGA, 'Deep': deep, 'Deep Allowed' : deep_allowed, 
                                     'scored': scored, 'conceded': missed, 'Expected Points': xpts, 'Result': result,
                                 'date': date, 'Wins': wins, 'Draws': draws, 'Loses': loses, 'Points': pts,
                                    'Non-penalty xG difference': npxGD, 'ppda attack': ppda_att, 
                                     'ppda defence': ppda_def, 'ppda allowed att': ppda_allowed_att,
                                    'ppda allowed def': ppda_allowed_def})
    if not os.path.exists('EPLteam2021_dfs'):
        os.makedirs('EPLteam2021_dfs')
    PL_Game_Data.to_csv('EPLteam2021_dfs/{}.csv'.format(team))

In [51]:
#Cumulative stats
for team in list(Table_data.keys()):
    df = pd.read_csv('EPLteam2021_dfs/{}.csv'.format(team))
    df['Wins cum'] = df['Wins'].cumsum()
    df['Loses cum'] = df['Loses'].cumsum()
    df['Draws cum'] = df['Draws'].cumsum()
    df['Points cum'] = df['Points'].cumsum()
    df['npxG difference cum'] = df['Non-penalty xG difference'].cumsum()
    df['xG cum'] = df['xG'].cumsum()
    df['xGA cum'] = df['xGA'].cumsum()
    df['npxG cum'] = df['non-penalty xG'].cumsum()
    df['npxGA cum'] = df['non-penalty xG Against'].cumsum()
    df['scored cum'] = df['scored'].cumsum()
    df['conceded cum'] = df['conceded'].cumsum()
    df['Expected Points cum'] = df['Expected Points'].cumsum()
    df.to_csv('EPLteam2021_dfs/{}.csv'.format(team))

In [52]:
#Use the Games (PL_Game_Data_2021) database to get opponent name, probability of win/lose/draw
for team in list(Table_data.keys()):
    df = pd.read_csv('EPLteam2021_dfs/{}.csv'.format(team))
    rows_to_drop = []
    for i in range(0, 380):
        if (PL_Game_Data_2021['Home Team'][i] != team and PL_Game_Data_2021['Away Team'][i] != team):
            rows_to_drop.append(i)
    Team_Games = PL_Game_Data_2021.drop(rows_to_drop)
    Team_Opps = []
    Prob_Win = []
    Prob_Draw = []
    Prob_Lose = []
    for i in list(Team_Games.index):
        if PL_Game_Data_1920['Home Team'][i] != team:
            Team_Opps.append(PL_Game_Data_2021['Home Team'][i])
            Prob_Lose.append(PL_Game_Data_2021['Probability Home Win'][i])
            Prob_Win.append(PL_Game_Data_2021['Probability Away Win'][i])
            Prob_Draw.append(PL_Game_Data_2021['Probability Draw'][i])
        else:
            Team_Opps.append(PL_Game_Data_2021['Away Team'][i])
            Prob_Win.append(PL_Game_Data_2021['Probability Home Win'][i])
            Prob_Lose.append(PL_Game_Data_2021['Probability Away Win'][i])
            Prob_Draw.append(PL_Game_Data_2021['Probability Draw'][i])
    df['Opponent'] = Team_Opps
    df['Prob Win'] = Prob_Win
    df['Prob Draw'] = Prob_Draw
    df['Prob Lose'] = Prob_Lose
    df.to_csv('EPLteam2021_dfs/{}.csv'.format(team))

In [53]:
#Getting stats for form of all the teams
for team in list(Table_data.keys()):
    df = pd.read_csv('EPLteam2021_dfs/{}.csv'.format(team))
    form_wins = []
    form_draws = []
    form_loses = []
    form_pts = []
    form_xg = []
    form_npxgdif = []
    form_xga = []
    form_npxg = []
    form_npxga = []
    form_scored = []
    form_conceded = []
    form_xpts = []
    for i in range(0,38):
        form_df = df[max(i-4, 0):i+1]
        form_wins.append(form_df['Wins'].cumsum()[i])
        form_draws.append(form_df['Draws'].cumsum()[i])
        form_loses.append(form_df['Loses'].cumsum()[i])
        form_pts.append(form_df['Points'].cumsum()[i])
        form_npxgdif.append(form_df['Non-penalty xG difference'].cumsum()[i])
        form_xg.append(form_df['xG'].cumsum()[i])
        form_xga.append(form_df['xGA'].cumsum()[i])
        form_npxg.append(form_df['non-penalty xG'].cumsum()[i])
        form_npxga.append(form_df['non-penalty xG Against'].cumsum()[i])
        form_scored.append(form_df['scored'].cumsum()[i])
        form_conceded.append(form_df['conceded'].cumsum()[i])
        form_xpts.append(form_df['Expected Points'].cumsum()[i])
    df['Wins in last 5'] = form_wins
    df['Loses in last 5'] = form_loses
    df['Draws in last 5'] = form_draws
    df['Points in last 5'] = form_pts
    df['npxG difference last 5'] = form_npxgdif
    df['xG last 5'] = form_xg
    df['xGA last 5'] = form_xga
    df['npxG last 5'] = form_npxg
    df['npxGA last 5'] = form_npxga
    df['scored last 5'] = form_scored
    df['conceded last 5'] = form_conceded
    df['Expected Points last 5'] = form_xpts
    df.to_csv('EPLteam2021_dfs/{}.csv'.format(team))

In [54]:
OG_cols = ['Home/Away',
 'xG',
 'xGA',
 'non-penalty xG',
 'non-penalty xG Against',
 'Deep',
 'Deep Allowed',
 'scored',
 'conceded',
 'Expected Points',
 'Result',
 'date',
 'Wins',
 'Draws',
 'Loses',
 'Points',
 'Non-penalty xG difference',
 'ppda attack',
 'ppda defence',
 'ppda allowed att',
 'ppda allowed def',
 'Wins cum',
 'Loses cum',
 'Draws cum',
 'Points cum',
 'npxG difference cum',
 'xG cum',
 'xGA cum',
 'npxG cum',
 'npxGA cum',
 'scored cum',
 'conceded cum',
 'Expected Points cum',
 'Opponent',
 'Prob Win',
 'Prob Draw',
 'Prob Lose',
 'Wins in last 5',
 'Loses in last 5',
 'Draws in last 5',
 'Points in last 5',
 'npxG difference last 5',
 'xG last 5',
 'xGA last 5',
 'npxG last 5',
 'npxGA last 5',
 'scored last 5',
 'conceded last 5',
 'Expected Points last 5']
Opp_cols = []
for col in OG_cols:
    Opp_cols.append('Opp ' + col)

In [56]:
for team in list(Table_data.keys()):
    df = pd.read_csv('EPLteam2021_dfs/{}.csv'.format(team))
    #Make first row have 0s coz the teams haven't played yet
    for col in list(df.columns):
        if 'Unnamed' in col:
            del df[col]
    df_ = pd.DataFrame(index = [0], columns=df.columns)
    df_ = df_.fillna(0)
    # Making a dataframe which has the OPP data from the previous gameweek - BUG HERE
    for i in range(1, len(list(df['Opponent']))):
        opp_df = pd.read_csv('EPLteam2021_dfs/{}.csv'.format(list(df['Opponent'])[i]))
        for col in list(opp_df.columns):
            if ('Unnamed' in col) or ('Opp ' in col):
                del opp_df[col]
        jeff = opp_df.iloc[i-1].to_frame()
        jeffT = jeff.T
        df_ = pd.concat([df_, jeffT], ignore_index=True)
    #Making new columns and adding them to the dataframe
    opp_dict = {OG_cols[i]: Opp_cols[i] for i in range(len(OG_cols))}
    df_2 = df_.rename(columns = opp_dict, inplace = False)
    df_3 = pd.concat([df, df_2], axis = 1)
    df_3.to_csv('EPLteam2021_dfs/{}.csv'.format(team))

Player Data

In [57]:
Player_ID = []
Player_name = []
Games = []
playtime_minutes = []
Goals = []
xG = []
Assists = []
xA = []
shots = []
key_passes = []
yellow_cards = []
red_cards = []
positions = []
team = []
nonpen_goals = []
nonpen_xG = []
xG_Chain = []
xG_Buildup = []
Player_data  = json.loads(Game_Table_Player_data[2].encode('ascii').decode('unicode-escape')[1:-1])
for Game in Player_data:
    Player_ID.append(list(Game.values())[0])
    Player_name.append(list(Game.values())[1])
    Games.append(list(Game.values())[2])
    playtime_minutes.append(list(Game.values())[3])
    Goals.append(list(Game.values())[4])
    xG.append(list(Game.values())[5])
    Assists.append(list(Game.values())[6])
    xA.append(list(Game.values())[7])
    shots.append(list(Game.values())[8])
    key_passes.append(list(Game.values())[9])
    yellow_cards.append(list(Game.values())[10])
    red_cards.append(list(Game.values())[11])
    positions.append(list(Game.values())[12])
    team.append(list(Game.values())[13])
    nonpen_goals.append(list(Game.values())[14])
    nonpen_xG.append(list(Game.values())[15])
    xG_Chain.append(list(Game.values())[16])
    xG_Buildup.append(list(Game.values())[17])
PL_Player_Data_2021 = pd.DataFrame({'id': Player_ID, 'Name': Player_name, 'Games Played': Games, 
                                  'Minutes Played': playtime_minutes, 'Goals': Goals, 'xG': xG,
                              'Assists' : Assists, 'xA': xA, 'Shots': shots, 'Key Passes': key_passes,
                                 'Yellow Cards': yellow_cards, 'Red Cards': red_cards, 'Positions': positions,
                                   'Team': team, 'Non Penalty Goals': nonpen_goals, 'Non Penalty xG': nonpen_xG,
                                   'xG Chain': xG_Chain, 'xG Buildup': xG_Buildup})

In [58]:
print(PL_Player_Data_2021.head(10))

     id                   Name Games Played Minutes Played Goals  \
0   647             Harry Kane           35           3097    23   
1  1250          Mohamed Salah           37           3085    22   
2  1228        Bruno Fernandes           37           3117    18   
3   453          Son Heung-Min           37           3139    17   
4   822        Patrick Bamford           38           3085    17   
5  5555  Dominic Calvert-Lewin           33           2878    16   
6   755            Jamie Vardy           34           2848    15   
7  8865          Ollie Watkins           37           3330    14   
8   314         Ilkay Gündogan           28           2033    13   
9  3277    Alexandre Lacazette           31           1939    13   

                   xG Assists                  xA Shots Key Passes  \
0  22.174858909100294      14   7.577093588188291   138         49   
1  20.250846637412906       5   6.528526101261377   126         55   
2    16.0194544615224      12  11.4749959